In [3]:
from pref_extract import find_restaurants
import pandas as pd

In [ ]:
restaurant_info = pd.read_csv('restaurant_info.csv')

In [ ]:
# Possible state transitions, not used, only to look at
# state_transition_possibilities = {
#     1: [2, 9],          
#     2: [2, 3, 9],
#     3: [3, 4, 9],
#     4: [4, 5, 9],
#     5: [6, 7, 9],
#     6: [5, 9],
#     7: [5, 8, 9],
#     8: [8, 9]
# }

In [ ]:
class StateMachine:
    def __init__(self, restaurant_info):
        self.state = 1
        self.preferences = {
            'food_type': None,
            'area': None,
            'price': None
        }
        self.restaurant_info = restaurant_info
        self.restaurants_options = pd.DataFrame()
        
        # TODO - add more messages
        self.message_dict = {
            1: 'Welcome to FeastFinder, I will find you a restaurant. Please tell what type of food would you like to eat, where, and in what price range',
            2: 'In what area should be the place?',
            3: 'What cuisine are you looking for?',
            4: 'In what price range should be the restaurant?',
            6: "I'm sorry, I don't see any restaurants matching your preferences. In my database. Try providing different preferences.",
            7: f'{restaurant_name} is located in {area} and is a {price} {food_type}. Does it sounds good?',
            8: f'{requested_info} of {restaurant_name} is {request_answer}'
            9: "I hope you'll have a great time. Bye bye!"
        }
    
    def change_state(self, utterance = None):
        next_state, if_message = self.predict_next_state(utterance)
        if if_message:
            print(self.message_dict[self.state])
            self.state = next_state
        else:
            self.state = next_state
            self.change_state()
    
    def predict_next_state(self, utterance = None) -> tuple(int, bool):
        """
        If the state should send a message, and then wait for user answer,
        it returns 'True'. If not, eg. state 2->4, returns False.
        :param utterance: 
        :return: tuple(next state number, if message sent) 
        """
        # TODO - category prediction, now - hardcoded category
        # category = predict(utterance)
        category = 'hello'
        if category == 'thankyou':
            return 9, True
        
        if self.state == 1:
            return 2, True
        
        if self.state == 2:
            if self.preferences['area'] is None:
                return 2, True
            else:
                return 3, False
            
        if self.state == 3:
            if self.preferences['food_type'] is None:
                return 3, True
            else:
                return 4, False
            
        if self.state == 4:
            if self.preferences['price'] is None:
                return 4, True
            else:
                return 5, False
        
        if self.state == 5:
            if (self.restaurants_options.empty):
                return 6, True
            else:
                return 7, True
        
        if self.state == 6:
            # TODO - extract new preferences
            # new_preference = extract_preference(utterance, ...)
            # self.preferences = new_preferences
            self.restaurants_options = find_restaurants(self.restaurant_info, self.preferences)
            return 5, False
            
        if self.state == 7:
            # TODO - category prediction, now - hardcoded category
            # category = predict(utterance)
            # category = 'request' / 'reqalts'
            category = 'request'
            if category == 'request':
                return 8, True
            if category == 'regalts':
                return 5, False
            # TODO: what happens else?
            #return ???
            
        if self.state == 8:
            return 8, True
        
        if self.state == 9:
            # TODO - its basically end of dialog
            return 9, True
            
        